# Hypothesis Testing 
Part 4

Jacob Tanzi

Using MySQL database from part 3

Answer questions for your stakeholder. 

#### *what makes a successful movie??*



#### Questions to Answer

#### Q1
* Does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?



#### Null Hypothesis
The MPAA rating of a movie does not affect the revenue generated
#### Alternative Hypothesis
The MPAA rating of a movie has a significant affect on the revenue generated

* Select the correct test according to the data type and number of samples
* Numerical Data
* Two groups rating and revenue
* Run a 2-sample T-test.

#### Q2
Do movies that are over 2 hours long earn more revenue than movies that are 1.5 hours long (or less)?
#### Null Hypothesis
There is no difference in earned revenue between movies that are over 2 hours long, than movies that are 1.5 hours long or less 
#### Alternative Hypothesis
There is a significant difference in earned revenue between movies that are over 2 hours long, than movies that are 1.5 hours long or less 


* Numerical Data
* Two groups runtime and revenue
* Run a 2-sample T-test.




#### Q3
Do some movie genres earn more revenue than others?
#### Null Hypothesis
The genre of a movie has no affect on revenue 

#### Alternative Hypothesis
The genre of a movie has a significant affect on revenue 



* Numerical Data
* Two groups revenue and genre(ohe)
* Run a 2-sample T-test.

In [1]:
import os, json, math, time
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
np.random.seed(321)

* They want you to perform a statistical test to get a mathematically-supported answer.
* They want you to report if you found a significant difference between ratings.
 * If so, what was the p-value of your analysis?
 * And which rating earns the most revenue?
 
* They want you to prepare a visualization that supports your finding.





#### Specifications
#### Your Data

* A critical first step for this assignment will be to retrieve additional movie data to add to your SQL database.
 * You will want to use the TMDB API again and extract data for additional years.
 * You may want to review the optional lesson from Week 1 on "Using Glob to Load Many Files" to load and combine all of your API results for each year.

* However, trying to extract the TMDB data for all movies from 2000-2022 could take >24 hours!

* To address this issue, you should EITHER:
 * Define a smaller (but logical) period of time to use for your analyses (e.g., last 10 years, 2010-2019 (pre-pandemic, etc).

 * OR coordinate with cohort-mates and divide the API calls so that you can all download the data for a smaller number of years and then share your downloaded JSON data.




In [2]:
FOLDER = "Data/"
file_list = sorted(os.listdir(FOLDER))
file_list

['.ipynb_checkpoints',
 '2010-2021',
 'Data',
 'TMDB api.ipynb',
 'combined_tmdb_data.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2010.csv.gz',
 'final_tmdb_data_2011.csv.gz',
 'final_tmdb_data_2012.csv.gz',
 'final_tmdb_data_2013.csv.gz',
 'final_tmdb_data_2014.csv.gz',
 'final_tmdb_data_2015.csv.gz',
 'final_tmdb_data_2016.csv.gz',
 'final_tmdb_data_2017.csv.gz',
 'final_tmdb_data_2018.csv.gz',
 'final_tmdb_data_2019.csv.gz',
 'final_tmdb_data_2020.csv.gz',
 'final_tmdb_data_2021.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2010.json',
 'tmdb_api_results_2011.json',
 'tmdb_api_results_2012.json',
 'tmdb_api_results_2013.json',
 'tmdb_api_results_2014.json',
 'tmdb_api_results_2015.json',
 'tmdb_api_results_2016.json',
 'tmdb_api_results_2017.json',
 'tmdb_api_results_2018.json',
 'tmdb_api_results_2019.json',
 'tmdb_api_res

In [3]:
q = FOLDER+"*.csv.gz"
print(q)

Data/*.csv.gz


In [4]:
file_list = glob.glob(q)
file_list

['Data\\combined_tmdb_data.csv.gz',
 'Data\\final_tmdb_data_2000.csv.gz',
 'Data\\final_tmdb_data_2001.csv.gz',
 'Data\\final_tmdb_data_2010.csv.gz',
 'Data\\final_tmdb_data_2011.csv.gz',
 'Data\\final_tmdb_data_2012.csv.gz',
 'Data\\final_tmdb_data_2013.csv.gz',
 'Data\\final_tmdb_data_2014.csv.gz',
 'Data\\final_tmdb_data_2015.csv.gz',
 'Data\\final_tmdb_data_2016.csv.gz',
 'Data\\final_tmdb_data_2017.csv.gz',
 'Data\\final_tmdb_data_2018.csv.gz',
 'Data\\final_tmdb_data_2019.csv.gz',
 'Data\\final_tmdb_data_2020.csv.gz',
 'Data\\final_tmdb_data_2021.csv.gz',
 'Data\\title_akas.csv.gz',
 'Data\\title_basics.csv.gz',
 'Data\\title_ratings.csv.gz',
 'Data\\tmdb_results_combined.csv.gz']

In [5]:
q = FOLDER+"*.csv.gz"
print(q)
file_list = sorted(glob.glob(q))
file_list

Data/*.csv.gz


['Data\\combined_tmdb_data.csv.gz',
 'Data\\final_tmdb_data_2000.csv.gz',
 'Data\\final_tmdb_data_2001.csv.gz',
 'Data\\final_tmdb_data_2010.csv.gz',
 'Data\\final_tmdb_data_2011.csv.gz',
 'Data\\final_tmdb_data_2012.csv.gz',
 'Data\\final_tmdb_data_2013.csv.gz',
 'Data\\final_tmdb_data_2014.csv.gz',
 'Data\\final_tmdb_data_2015.csv.gz',
 'Data\\final_tmdb_data_2016.csv.gz',
 'Data\\final_tmdb_data_2017.csv.gz',
 'Data\\final_tmdb_data_2018.csv.gz',
 'Data\\final_tmdb_data_2019.csv.gz',
 'Data\\final_tmdb_data_2020.csv.gz',
 'Data\\final_tmdb_data_2021.csv.gz',
 'Data\\title_akas.csv.gz',
 'Data\\title_basics.csv.gz',
 'Data\\title_ratings.csv.gz',
 'Data\\tmdb_results_combined.csv.gz']

In [6]:
q = FOLDER+"/**/*.csv.gz"
print(q)
file_list = sorted(glob.glob(q,recursive=True))
file_list

Data//**/*.csv.gz


['Data\\combined_tmdb_data.csv.gz',
 'Data\\final_tmdb_data_2000.csv.gz',
 'Data\\final_tmdb_data_2001.csv.gz',
 'Data\\final_tmdb_data_2010.csv.gz',
 'Data\\final_tmdb_data_2011.csv.gz',
 'Data\\final_tmdb_data_2012.csv.gz',
 'Data\\final_tmdb_data_2013.csv.gz',
 'Data\\final_tmdb_data_2014.csv.gz',
 'Data\\final_tmdb_data_2015.csv.gz',
 'Data\\final_tmdb_data_2016.csv.gz',
 'Data\\final_tmdb_data_2017.csv.gz',
 'Data\\final_tmdb_data_2018.csv.gz',
 'Data\\final_tmdb_data_2019.csv.gz',
 'Data\\final_tmdb_data_2020.csv.gz',
 'Data\\final_tmdb_data_2021.csv.gz',
 'Data\\title_akas.csv.gz',
 'Data\\title_basics.csv.gz',
 'Data\\title_ratings.csv.gz',
 'Data\\tmdb_results_combined.csv.gz']

In [7]:
df = pd.concat([pd.read_csv(f) for f in file_list])
df

C:\Users\Jacob\AppData\Local\Temp\ipykernel_16000\3559116000.py:1: DtypeWarning: Columns (0,2,3,5,6,8,9,10,12,13,14,15,18,19,20,21,25,27,28,30,31,32,33,35,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv(f) for f in file_list])


,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,averageRating,numVotes
0,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,843.0,cn,花樣年華,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0118852,0.0,/vceiGZ3uavAEHlTA7v0GjQsGVKe.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,49511.0,en,Chinese Coffee,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2593,tt7797790,0.0,NaN,NaN,0.0,"[{'id': 27, 'name': 'Horror'}]",NaN,956219.0,en,Edmund Kemper Part 3: La mort sévit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2594,tt8665056,0.0,NaN,NaN,0.0,"[{'id': 37, 'name': 'Western'}]",http://skeletoncreekproductions.com/p-movie-br...,885436.0,en,Guns Along The Bravo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2595,tt8795764,0.0,NaN,NaN,0.0,"[{'id': 27, 'name': 'Horror'}]",https://www.utahwolf.com/films/coming-soon-new...,871624.0,en,New Breed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2596,tt9071078,0.0,NaN,NaN,0.0,"[{'id': 28, 'name': 'Action'}]",NaN,201706.0,cn,致命密函,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8200004 entries, 0 to 2597
Data columns (total 45 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   imdb_id                object 
 1   adult                  float64
 2   backdrop_path          object 
 3   belongs_to_collection  object 
 4   budget                 float64
 5   genres                 object 
 6   homepage               object 
 7   id                     float64
 8   original_language      object 
 9   original_title         object 
 10  overview               object 
 11  popularity             float64
 12  poster_path            object 
 13  production_companies   object 
 14  production_countries   object 
 15  release_date           object 
 16  revenue                float64
 17  runtime                float64
 18  spoken_languages       object 
 19  status                 object 
 20  tagline                object 
 21  title                  object 
 22  video                

In [9]:
#remove 0's
df = df.loc[ df['imdb_id']!='0']
df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,averageRating,numVotes
0,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,843.0,cn,花樣年華,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0118852,0.0,/vceiGZ3uavAEHlTA7v0GjQsGVKe.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,49511.0,en,Chinese Coffee,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2593,tt7797790,0.0,NaN,NaN,0.0,"[{'id': 27, 'name': 'Horror'}]",NaN,956219.0,en,Edmund Kemper Part 3: La mort sévit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2594,tt8665056,0.0,NaN,NaN,0.0,"[{'id': 37, 'name': 'Western'}]",http://skeletoncreekproductions.com/p-movie-br...,885436.0,en,Guns Along The Bravo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2595,tt8795764,0.0,NaN,NaN,0.0,"[{'id': 27, 'name': 'Horror'}]",https://www.utahwolf.com/films/coming-soon-new...,871624.0,en,New Breed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2596,tt9071078,0.0,NaN,NaN,0.0,"[{'id': 28, 'name': 'Action'}]",NaN,201706.0,cn,致命密函,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#reset index
df = df.reset_index(drop=True)
df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,averageRating,numVotes
0,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,843.0,cn,花樣年華,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0118852,0.0,/vceiGZ3uavAEHlTA7v0GjQsGVKe.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,49511.0,en,Chinese Coffee,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8199983,tt7797790,0.0,NaN,NaN,0.0,"[{'id': 27, 'name': 'Horror'}]",NaN,956219.0,en,Edmund Kemper Part 3: La mort sévit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8199984,tt8665056,0.0,NaN,NaN,0.0,"[{'id': 37, 'name': 'Western'}]",http://skeletoncreekproductions.com/p-movie-br...,885436.0,en,Guns Along The Bravo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8199985,tt8795764,0.0,NaN,NaN,0.0,"[{'id': 27, 'name': 'Horror'}]",https://www.utahwolf.com/films/coming-soon-new...,871624.0,en,New Breed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8199986,tt9071078,0.0,NaN,NaN,0.0,"[{'id': 28, 'name': 'Action'}]",NaN,201706.0,cn,致命密函,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8199988 entries, 0 to 8199987
Data columns (total 45 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   imdb_id                object 
 1   adult                  float64
 2   backdrop_path          object 
 3   belongs_to_collection  object 
 4   budget                 float64
 5   genres                 object 
 6   homepage               object 
 7   id                     float64
 8   original_language      object 
 9   original_title         object 
 10  overview               object 
 11  popularity             float64
 12  poster_path            object 
 13  production_companies   object 
 14  production_countries   object 
 15  release_date           object 
 16  revenue                float64
 17  runtime                float64
 18  spoken_languages       object 
 19  status                 object 
 20  tagline                object 
 21  title                  object 
 22  video             

## Preprocessing 

In [13]:
## Columns to exclude
drop_cols = ['backdrop_path','backdrop_path','original_title','overview',
                 'poster_path','tagline','id','homepage',
                 'production_countries','video','spoken_languages',
            'original_language']
df = df.drop(columns=drop_cols)
df

MemoryError: Unable to allocate 375. MiB for an array with shape (6, 8199988) and data type float64

MemoryError: Unable to allocate 313. MiB for an array with shape (5, 8199988) and data type float64

## Feature Engineering

* Collection to boolean
* Pull out Genre names then OHE
* Cleaning
* Converting release date to year, month

In [ ]:
df['belongs_to_collection'].notna().sum()

In [ ]:
# Use .notna() to get True if it belongs to a collection
df['belongs_to_collection'] = df['belongs_to_collection'].notna()
df['belongs_to_collection'].value_counts()

In [ ]:
df.info()

In [ ]:
df['genres'][0]

In [ ]:
def get_genre_name(x):
    if isinstance(x, str):
        x = x.replace("'", '"')
        x = json.loads(x)
        genres = []
        for item in x:
            genres.append(item['name'])
        return genres
    return []


In [ ]:
def get_genre_name(x):
    if isinstance(x, str):
        try:
            x = x.replace("'", '"')
            x = json.loads(x)
            genres = []
            for item in x:
                genres.append(item['name'])
            return genres
        except json.JSONDecodeError:
            return []  # Return an empty list if JSON decoding fails
    return []

In [ ]:
get_genre_name(df.loc[3,'genres'])

In [ ]:
df['genres_list'] = df['genres'].apply(get_genre_name)
df_explode = df.explode('genres_list')
df_explode.head()

In [ ]:
## save unique genres
unique_genres = df_explode['genres_list'].dropna().unique()
unique_genres

In [ ]:
## Manually One-Hot-Encode Genres
for genre in unique_genres:
    df[f"Genre_{genre}"] = df['genres'].str.contains(genre,regex=False)    
df

In [ ]:
## Drop original genre cols
df = df.drop(columns=['genres','genres_list'])
df

In [ ]:
df.info()

In [ ]:
## Checking Certification values
df['certification'].value_counts(dropna=False)

In [ ]:
# fix extra space certs
df['certification'] = df['certification'].str.strip()

In [ ]:
## fix certification col
repl_cert = {'Unrated':'NR',}

df['certification'] = df['certification'].replace(repl_cert)
df['certification'].value_counts(dropna=False)

In [ ]:
## split release date into 3 columns
new_cols = ['year','month','day']
df[new_cols] = df['release_date'].str.split('-',expand=True)
df[new_cols] = df[new_cols].astype(float)
df

In [ ]:
df[df['release_date'].isna()]

In [ ]:
## drop original feature
df = df.drop(columns=['release_date'])

In [ ]:
df.info()

In [ ]:
## Check status
df['status'].value_counts()

In [ ]:
## Save only Released status
df = df.loc[ df['status'] == 'Released']
df = df.drop(columns=['status'])
df

In [ ]:
## filter out financials that don't have financial data
df = df.loc[(df['budget'] >0 ) & (df['revenue']>0)]
df

## Train Test

In [ ]:
df.info()

In [ ]:
drop_for_model = ['title','imdb_id','production_companies']
df = df.drop(columns=drop_for_model)
df

In [ ]:
## Make x and y variables
y = df['revenue'].copy()
X = df.drop(columns=['revenue']).copy()

X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train.head()

In [ ]:
X_train.isna().sum()

In [ ]:
## make cat selector and using it to save list of column names
cat_select = make_column_selector(dtype_include='object')
cat_cols = cat_select(X_train)
cat_cols

In [ ]:
## select manually OHE cols for later
bool_select = make_column_selector(dtype_include='bool')
already_ohe_cols = bool_select(X_train)
already_ohe_cols

In [ ]:
num_select = make_column_selector(dtype_include='number')
num_cols = num_select(X_train)
num_cols

In [ ]:
## convert manual ohe to int
X_train[already_ohe_cols] = X_train[already_ohe_cols].astype(int)
X_test[already_ohe_cols] = X_test[already_ohe_cols].astype(int)

## Pipelines

In [ ]:

cat_pipe = make_pipeline(SimpleImputer(strategy='constant',
                                       fill_value='MISSING'),
                         OneHotEncoder(handle_unknown='ignore', sparse=False))
num_pipe = make_pipeline(SimpleImputer(strategy='mean'),#StandardScaler()
                        )

preprocessor = make_column_transformer((cat_pipe,cat_cols),
                                        (num_pipe, num_cols),
                                       ('passthrough',already_ohe_cols))# remainder='passthrough')
preprocessor

In [ ]:
xx123123= h 

In [ ]:
## fit the col transformer
preprocessor.fit(X_train)

## Finding the categorical pipeline in our col transformer.
preprocessor.named_transformers_['pipeline-1']

In [ ]:
## B) Using list-slicing to find the encoder 
cat_features = preprocessor.named_transformers_['pipeline-1'][-1].get_feature_names_out(cat_cols)


## Create the empty list
final_features = [*cat_features,*num_cols,*already_ohe_cols]
len(final_features)

In [ ]:
preprocessor.transform(X_train).shape

In [ ]:
X_train_tf = pd.DataFrame( preprocessor.transform(X_train), 
                          columns=final_features, index=X_train.index)
X_train_tf.head()

In [ ]:
X_test_tf = pd.DataFrame( preprocessor.transform(X_test),
                         columns=final_features, index=X_test.index)
X_test_tf.head()

## Statsmodels

In [ ]:
##import statsmodels correctly
import statsmodels.api as sm

In [ ]:
# Tip: make sure that add_constant actually added a new column! 
#You may need to change the parameter has_constant to "add"

In [ ]:
## Make final X_train_df and X_test_df with constants added
X_train_df = sm.add_constant(X_train_tf, prepend=False)
X_test_df = sm.add_constant(X_test_tf, prepend=False)
display(X_train_df.head(2),X_test_df.head(2))

## Modeling
#### Statsmodel

In [ ]:
## instantiate an OLS model WITH the training data.
model = sm.OLS(y_train, X_train_df)

## Fit the model and view the summary
result = model.fit()
result.summary()

In [ ]:
## Get train data performance from skearn to confirm matches OLS
y_hat_train = result.predict(X_train_df)
print(f'Training R^2: {r2_score(y_train, y_hat_train):.3f}')

## Get test data performance
y_hat_test = result.predict(X_test_df)
print(f'Testing R^2: {r2_score(y_test, y_hat_test):.3f}')

## Linear Regression

#### QQ-Plot for Checking for Normality

In [ ]:
## Create a Q-QPlot

# first calculate residuals 
resid = y_test - y_hat_test

## then use sm's qqplot
fig, ax = plt.subplots(figsize=(6,4))
sm.graphics.qqplot(resid,line='45',fit=True,ax=ax);

In [ ]:
#Residual Plot for Checking Homoscedasticity
## Plot scatterplot with y_hat_test vs resids
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(y_hat_test, resid, ec='white')
ax.axhline(0,c='black',zorder=0)
ax.set(ylabel='Residuals',xlabel='Predicted Revenue')

## Evaluate 

In [ ]:
def evaluate_ols(result,X_train_df, y_train):
    """Plots a Q-Q Plot and residual plot for a statsmodels OLS regression.
    """
    
    ## save residuals from result
    y_pred = result.predict(X_train_df)
    resid = y_train - y_pred
    
    fig, axes = plt.subplots(ncols=2,figsize=(12,5))
    
    ## Normality 
    sm.graphics.qqplot(resid,line='45',fit=True,ax=axes[0]);
    
    ## Homoscedasticity
    ax = axes[1]
    ax.scatter(y_pred, resid, edgecolor='white',lw=1)
    ax.axhline(0,zorder=0)
    ax.set(ylabel='Residuals',xlabel='Predicted Value');
    plt.tight_layout()
    

In [ ]:
evaluate_ols(result,X_train_df, y_train)

#### Questions to Answer

#### Q1
* Does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?



#### Null Hypothesis
The MPAA rating of a movie does not affect the revenue generated
#### Alternative Hypothesis
The MPAA rating of a movie has a significant affect on the revenue generated

* Select the correct test according to the data type and number of samples
* Numerical Data
* More than two groups genres revenue 
* Run ANOVA


In [ ]:
df['certification'].describe()

In [ ]:
df['revenue'].describe()

In [ ]:
df['certification'].value_counts()

In [ ]:
ax = sns.barplot(data=df, x='certification', y='revenue')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45);

In [ ]:
need_cols = ['certification', 'revenue']
df[need_cols]

In [ ]:
groups ={}

for certification in df['certification'].unique():
    temp = df.loc[df['certification']== certification, 'revenue']
    groups[certification] = temp

groups.keys()

In [ ]:
groups['XXXXXXX']

In [ ]:
for certification, data in groups.items():

    ## determine if there are any outliers
    outliers = np.abs(stats.zscore(data)) > 3
    
    ## print a statement about how many outliers for which group name
    print(f"There were {outliers.sum()} outliers in the {certification} group.")

    ## Remove the outiers from data and overwrite the sector data in the dict
    data = data.loc[~outliers]
    groups[certification] = data

In [ ]:
norm_results = [['group','n','pval','sig?']]

## loop through group dict
for certification, data in groups.items():
    ## calculate normaltest results
    stat, p = stats.normaltest(data)
    
    ## Append the right info into norm_resutls (as a list)
    norm_results.append(certification,len(data), p, p<.05])
    
    
## Make norm_results a dataframe (first row is columns, everything else data)
normal_results = pd.DataFrame(norm_results[1:], columns = norm_results[0])
normal_results

In [ ]:
result = stats.levene(groups['XXXX'],
            groups['XXXX'],
            groups['XXXX'],
            groups['XXXX'])

In [ ]:
result = stats.levene(*groups.values())
print(result)

In [ ]:
if result.pvalue < .05:
    print(f"The groups do NOT have equal variance.")
else:
    print(f"The groups DO have equal variance.")

In [ ]:
result = stats.kruskal(*groups.values())
print(result)
result.pvalue<.05

In [ ]:
average_revenue_by_certification = df.groupby('certification')['revenue'].mean().reset_index()

overall_average_revenue = df['revenue'].mean()
std = df['revenue'].std()

plt.figure(figsize=(10, 6))

ax = sns.barplot(data=average_charges_by_certification, x='certification', y='revenue')

plt.axhline(y=overall_average_revenue, color='red', linestyle='--', label='Overall Average Charges')
plt.axhline(overall_average_revenue + std, color='black', linestyle='dotted', label='+1 Std')
plt.axhline(overall_average_revenue - std, color='black', linestyle='dotted', label='-1 Std')

plt.axvspan(-0.5, len(average_revenue_by_certification) - 0.5, ymin=(overall_average_revenue - std) / ax.get_ylim()[1],
            ymax=(overall_average_revenue + std) / ax.get_ylim()[1], color='yellow', alpha=0.3, label='+/- 1 Std Range')


ax.set(ylabel='Average Revenue')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '${:,.0f}'.format(x)))

plt.title('Average Revenue by Rating Compared to Overall Average Charges')
plt.xlabel('Region')
plt.xticks(rotation=45)
plt.tight_layout()
plt.legend()

plt.show()

#### Q2
Do movies that are over 2 hours long earn more revenue than movies that are 1.5 hours long (or less)?
#### Null Hypothesis
There is no difference in earned revenue between movies that are over 2 hours long, than movies that are 1.5 hours long or less 
#### Alternative Hypothesis
There is a significant difference in earned revenue between movies that are over 2 hours long, than movies that are 1.5 hours long or less 


* Numerical Data
* Two groups runtime and revenue
* Run a 2-sample T-test.




In [ ]:
df['revenue'].describe()

In [ ]:
df['runtime'].describe()

In [ ]:
df['over_2_hour'] = df['runtime']> 120
df['over_2_hour'].value_counts()

In [ ]:
needed_cols = ['over_2_hour','revenue']

df[needed_cols]

needed_cols = ['over_2_hour', 'revenue']

# Display rows where 'over_2_hour' is True along with 'revenue' column
selected_rows = df.loc[df['over_2_hour'], needed_cols]
print(selected_rows)

In [ ]:
over_2_hour_df = df.loc[ df['over_2_hour']==True, needed_cols]
over_2_hour_df

In [ ]:
df['under_90min'] = df['runtime']< 90
df['under_90min'].value_counts()

In [ ]:
under_90min_df = df.loc[ df['under_90min']==True, needed_cols]
under_90min_df

In [ ]:
plot_df = pd.concat([under_90min_df, over_2_hour_df])
plot_df

In [ ]:
sns.barplot(data=plot_df, x='under_90min',y='revenue');

In [ ]:
under_90min_group = under_90min_df['revenue']
over_2_hour_group = over_2_hour_df['revenue']

In [ ]:
u90min_outliers = np.abs(stats.zscore(under_90min_group)) > 3
u90min_outliers.sum()

In [ ]:
o2hr_outliers = np.abs(stats.zscore(over_2_hour_group)) > 3
o2hr_outliers.sum()

In [ ]:
result = stats.levene(under_90min_group, over_2_hour_group)
print(result)

In [ ]:
if result.pvalue < .05:
    print(f"The groups do NOT have equal variance.")
else:
    print(f"The groups DO have equal variance.")

In [ ]:
result = stats.ttest_ind(under_90min_group, over_2_hour_group, equal_var=False)
print(result)
result.pvalue  < .05

In [ ]:
under_90min_group.mean()

In [ ]:
over_2_hour_group.mean()

In [ ]:
plt.figure(figsize=(10, 6))
ax = sns.histplot(data=plot_df, y='revenue',fill=True,hue='under_90min',kde=True,
                  stat='density',common_norm=False)
ax.ticklabel_format(style='plain', axis='y')
plt.title('Revenue($) vs Runtime(90min +/-)', fontsize='xx-large');

#### Q3
Do some movie genres earn more revenue than others?
#### Null Hypothesis
The genre of a movie has no affect on revenue 

#### Alternative Hypothesis
The genre of a movie has a significant affect on revenue 



* Numerical Data
* Two groups revenue and genre(ohe)
* Run a 2-sample T-test.

In [ ]:
## save unique genres
unique_genres = df_explode['genres_list'].dropna().unique()
unique_genres

## Improving Model

## Checking for Linearity

In [ ]:
## concatenating training data into plot_df
plot_df = pd.concat([X_train_df,y_train],axis=1)
plot_df

In [ ]:
## save plot_cols list to show (dropping genre from plot_df from pair_plot)
genre_cols = [c for c in df.columns if c.startswith('Genre')]
plot_cols = plot_df.drop(columns=['revenue',*genre_cols]).columns
len(plot_cols)

In [ ]:
sns.pairplot(data=plot_df, y_vars='revenue',x_vars=plot_cols[:6])

In [ ]:
sns.pairplot(data=plot_df, y_vars='revenue',x_vars=plot_cols[6:13])

In [ ]:
sns.pairplot(data=plot_df, y_vars='revenue',x_vars=plot_cols[13:])

In [ ]:
# remove movies prior to 2000
df = df.loc[ df['year']>=2000]
df

In [ ]:
def get_train_test_split(df_, y_col='revenue',drop_cols=[]):
    
    ## Make copy of input df
    df = df_.copy()
    
    ## filter columns in drop cols (if exist)
    final_drop_cols = []
    [df.drop(columns=c,inplace=True) for c in df.columns if c in drop_cols]
    
    
    ## Make x and y variables
    y = df[y_col].copy()
    X = df.drop(columns=[y_col]).copy()

    X_train, X_test, y_train, y_test = train_test_split(X,y)#, random_state=321)
    

    
    ## make cat selector and using it to save list of column names
    cat_select = make_column_selector(dtype_include='object')
    cat_cols = cat_select(X_train)


    ## make num selector and using it to save list of column names
    num_select = make_column_selector(dtype_include='number')
    num_cols = num_select(X_train)


    ## select manually OHE cols for later
    bool_select = make_column_selector(dtype_include='bool')
    already_ohe_cols = bool_select(X_train)

    ## convert manual ohe to int
    X_train[already_ohe_cols] = X_train[already_ohe_cols].astype(int)
    X_test[already_ohe_cols] = X_test[already_ohe_cols].astype(int)

    ## make pipelines
    cat_pipe = make_pipeline(SimpleImputer(strategy='constant',
                                           fill_value='MISSING'),
                             OneHotEncoder(handle_unknown='ignore', sparse=False))
    num_pipe = make_pipeline(SimpleImputer(strategy='mean'),#StandardScaler()
                            )

    preprocessor = make_column_transformer((num_pipe, num_cols),
                                           (cat_pipe,cat_cols),remainder='passthrough')
    
    

    ## fit the col transformer
    preprocessor.fit(X_train)

    ## Finding the categorical pipeline in our col transformer.
    preprocessor.named_transformers_['pipeline-2']

    ## B) Using list-slicing to find the encoder 
    cat_features = preprocessor.named_transformers_['pipeline-2'][-1].get_feature_names_out(cat_cols)


    ## Create the empty list
    final_features = [*cat_features,*num_cols,*already_ohe_cols]

    ## Make df verisons of x data
    X_train_tf = pd.DataFrame( preprocessor.transform(X_train), 
                              columns=final_features, index=X_train.index)


    X_test_tf = pd.DataFrame( preprocessor.transform(X_test),
                             columns=final_features, index=X_test.index)


    ### Adding a Constant for Statsmodels
    ## Make final X_train_df and X_test_df with constants added
    X_train_df = sm.add_constant(X_train_tf, prepend=False, has_constant='add')
    X_test_df = sm.add_constant(X_test_tf, prepend=False, has_constant='add')
    return X_train_df, y_train, X_test_df, y_test

In [ ]:
## Use our function to make new x,y vars
X_train_df, y_train, X_test_df, y_test = get_train_test_split(df)

## instantiate an OLS model WITH the training data.
model = sm.OLS(y_train, X_train_df)

## Fit the model and view the summary
result = model.fit()
evaluate_ols(result,X_train_df,y_train)

In [ ]:
sns.boxplot(x = y_train)

## Removing Outliers
#### Z score

In [ ]:
def find_outliers(data, verbose=True):
    outliers = np.abs(stats.zscore(data))>3
    
    if verbose:
        print(f"- {outliers.sum()} outliers found in {data.name} using Z-Scores.")
    return outliers

In [ ]:
def find_outliers_IQR(data, verbose=True):
    q3 = np.quantile(data,.75)
    q1 = np.quantile(data,.25)

    IQR = q3 - q1
    upper_threshold = q3 + 1.5*IQR
    lower_threshold = q1 - 1.5*IQR
    
    outliers = (data<lower_threshold) | (data>upper_threshold)
    if verbose:
        print(f"- {outliers.sum()} outliers found in {data.name} using IQR.")
        
    return outliers

In [ ]:
outliers_iqr = find_outliers_IQR(df['revenue'])
outliers_iqr

In [ ]:
## Loop to remove outliers from same clumns using new function
outlier_cols = ['runtime','popularity','revenue']

## Empty dict for both types of outliers
outliers_z = {}
outliers_iqr = {}

## Use both functions to see the comparison for # of outliers
for col in outlier_cols:
    outliers_col_z = find_outliers(df[col])
    outliers_z[col] = outliers_col_z
    
    outliers_col_iqr = find_outliers_IQR(df[col])
    outliers_iqr[col] = outliers_col_iqr
    print()
    

In [ ]:
# remove_outliers 
df_clean_z = df.copy()
for col, idx_outliers in outliers_z.items():
    df_clean_z = df_clean_z[~idx_outliers]
df_clean_z

In [ ]:
# remove_outliers
df_clean_iqr = df.copy()
for col, idx_outliers in outliers_iqr.items():
    df_clean_iqr = df_clean_iqr[~idx_outliers]
df_clean_iqr

## Model with Outliers Removed  
#### (Z_scores)

In [ ]:
## MAKE NEW MODEL WITH IQR OUTLIERS REMOVED

X_train_df, y_train, X_test_df, y_test = get_train_test_split(df_clean_iqr)
## instantiate an OLS model WITH the training data.
model = sm.OLS(y_train, X_train_df)

## Fit the model and view the summary
result = model.fit()
display(result.summary())
evaluate_ols(result,X_train_df,y_train)

## Pvalues

In [ ]:
pvals = result.pvalues 
pvals

In [ ]:
pvals[ pvals>.05]

## Feature engineering by p-values

In [ ]:
## Get list of ALL genre columns to see how many are sig
genre_cols = [c for c in df_clean.columns if c.startswith("Genre")]
genre_cols

In [ ]:
## save just genre pvalues
genre_pvals = pvals[genre_cols]
## calc what % are insig?
genre_pvals.sum()/len(genre_pvals)

In [ ]:
## what pvals are remaining?
other_pvals = pvals.drop([*comp_cols, *genre_cols])
other_pvals[other_pvals>.05]

In [ ]:
## Make x,y vars, but drop features with bad pvalues
df_clean_iqr = df_clean_iqr.drop(columns=['adult','budget'])
X_train_df, y_train, X_test_df, y_test = \
                                get_train_test_split(df_clean_iqr,)
## MAKE AND EVALUATE OLS
## instantiate an OLS model WITH the training data.
model = sm.OLS(y_train, X_train_df)

## Fit the model and view the summary
result = model.fit()
evaluate_ols(result,X_train_df,y_train)

## Multicollinearity

In [ ]:
## Calculating the mask to hide the upper-right of the triangle
plt.figure(figsize=(25,25))
corr = X_train_df.corr().abs()
mask = np.triu(np.ones_like(corr))
sns.heatmap(corr,square=True, cmap='Reds', annot=True, mask=mask);

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
 
# separate just x-data and subtract mean
features = X_train_df -  X_train_df.mean()

features

In [ ]:
# create a list of VIF scores for each feature in features.
vif_scores = [variance_inflation_factor(features.values, i) for i in range(len(features.columns))]

# create a new dataframe to hold the VIF scores 
VIF = pd.Series(vif_scores, index=features.columns)
VIF

In [ ]:
## Sort VIF for inspect
VIF.sort_values()

In [ ]:
## Set float format to view vals not in scientfic notation
pd.set_option('display.float_format',lambda x: f'{x:.2f}')

In [ ]:
## remove infinite values from VIF and sort
VIF = VIF[VIF!=np.inf].sort_values()
VIF

In [ ]:
## filter for VIF that are > 5
VIF[VIF>5]

In [ ]:
high_vif = VIF[VIF>5].index
high_vif

## Final Model

In [ ]:
## make new df_final copy of prev df
df_final = df_clean_iqr.drop(columns =high_vif).copy()
X_train_df, y_train, X_test_df, y_test = get_train_test_split(df_final)
## instantiate an OLS model WITH the training data.
model = sm.OLS(y_train, X_train_df)

## Fit the model and view the summary
result = model.fit()
evaluate_ols(result,X_train_df,y_train)

In [ ]:
## Get train data performance from skearn to confirm matches OLS
y_hat_train = result.predict(X_train_df)
print(f'Training R^2: {r2_score(y_train, y_hat_train):.3f}')

## Get test data performance
y_hat_test = result.predict(X_test_df)
print(f'Testing R^2: {r2_score(y_test, y_hat_test):.3f}')

In [ ]:
plt.figure(figsize=(5,16))
ax =result.params.sort_values().plot(kind='barh')
ax.axvline()

## Linear Reg vs RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
X_train_df, y_train, X_test_df, y_test = get_train_test_split(df)

reg = RandomForestRegressor(verbose=1,random_state=42)
reg.fit(X_train_df, y_train)

In [ ]:
## Get train data performance from skearn to confirm matches OLS
y_hat_train = reg.predict(X_train_df)
print(f'Training R^2: {r2_score(y_train, y_hat_train):.3f}')

## Get test data performance
y_hat_test = reg.predict(X_test_df)
print(f'Testing R^2: {r2_score(y_test, y_hat_test):.3f}')

In [ ]:
evaluate_ols(reg, X_train_df, y_train)

In [ ]:
importances = pd.Series(reg.feature_importances_, index=X_train_df.columns)
importances.sort_values().tail(25).plot(kind='barh',figsize=(6,10))
